# Teamcenter AI Agent Setup Testing

This notebook tests your setup step-by-step safely.

**Important**: Run cells one by one!

## Step 1: Configuration Test

In [ ]:
print("=" * 60)
print("STEP 1: CONFIGURATION LOADING")
print("=" * 60)

try:
    from config import Config
    print("✓ Configuration module imported successfully")
    
    print(f"\n📁 Project Configuration:")
    print(f"   Project Root: {Config.PROJECT_ROOT}")
    print(f"   Data Directory: {Config.DATA_DIR}")
    print(f"   Documents Directory: {Config.DOCUMENTS_DIR}")
    
    print(f"\n🤖 AI Model Configuration:")
    print(f"   Primary Model: {Config.PRIMARY_MODEL}")
    print(f"   Embedding Model: {Config.EMBEDDING_MODEL}")
    print(f"   Max Tokens: {Config.MAX_TOKENS}")
    print(f"   Temperature: {Config.TEMPERATURE}")
    
    print(f"\n🔧 RAG Configuration:")
    print(f"   Chunk Size: {Config.CHUNK_SIZE}")
    print(f"   Chunk Overlap: {Config.CHUNK_OVERLAP}")
    print(f"   Top-K Retrieval: {Config.TOP_K_RETRIEVAL}")
    
    # Check API keys with more detail
    print(f"\n🔑 API Key Status:")
    
    if Config.ANTHROPIC_API_KEY:
        key_preview = Config.ANTHROPIC_API_KEY[:20] + "..." + Config.ANTHROPIC_API_KEY[-10:]
        print(f"   Anthropic: ✓ SET ({len(Config.ANTHROPIC_API_KEY)} chars) - {key_preview}")
    else:
        print(f"   Anthropic: ✗ MISSING")
        
    if Config.LANGFUSE_PUBLIC_KEY and Config.LANGFUSE_SECRET_KEY:
        pub_preview = Config.LANGFUSE_PUBLIC_KEY[:15] + "..."
        sec_preview = Config.LANGFUSE_SECRET_KEY[:15] + "..."
        print(f"   Langfuse Public: ✓ SET ({len(Config.LANGFUSE_PUBLIC_KEY)} chars) - {pub_preview}")
        print(f"   Langfuse Secret: ✓ SET ({len(Config.LANGFUSE_SECRET_KEY)} chars) - {sec_preview}")
        print(f"   Langfuse Host: {Config.LANGFUSE_HOST}")
    else:
        print(f"   Langfuse: ✗ MISSING")
        
    if Config.OPENAI_API_KEY:
        openai_preview = Config.OPENAI_API_KEY[:20] + "..." + Config.OPENAI_API_KEY[-10:]
        print(f"   OpenAI: ✓ SET ({len(Config.OPENAI_API_KEY)} chars) - {openai_preview}")
    else:
        print(f"   OpenAI: ○ NOT SET (optional)")
        
    if Config.GOOGLE_API_KEY:
        google_preview = Config.GOOGLE_API_KEY[:20] + "..."
        print(f"   Google: ✓ SET ({len(Config.GOOGLE_API_KEY)} chars) - {google_preview}")
    else:
        print(f"   Google: ○ NOT SET (optional)")
    
    # Validate configuration
    print(f"\n🔍 Configuration Validation:")
    errors = Config.validate_config()
    if errors:
        print("   ⚠️ Issues found:")
        for i, error in enumerate(errors, 1):
            print(f"      {i}. {error}")
    else:
        print("   ✓ All validation checks passed!")
        
    print(f"\n✅ CONFIGURATION TEST: PASSED")
        
except Exception as e:
    print(f"❌ CONFIGURATION TEST: FAILED")
    print(f"Error: {e}")
    print(f"\nFull traceback:")
    import traceback
    traceback.print_exc()

## Step 2: Dependencies Test

In [ ]:
print("Testing core dependencies...")

dependencies = [
    ('anthropic', 'Anthropic Claude API'),
    ('langfuse', 'Langfuse experiment tracking'),
    ('sentence_transformers', 'Sentence Transformers'),
    ('faiss', 'FAISS vector database'),
    ('numpy', 'NumPy'),
    ('pandas', 'Pandas')
]

success_count = 0
for package, description in dependencies:
    try:
        if package == 'faiss':
            import faiss
        else:
            __import__(package)
        print(f"SUCCESS: {description}")
        success_count += 1
    except ImportError as e:
        print(f"FAILED: {description}: {e}")

print(f"\nResult: {success_count}/{len(dependencies)} dependencies OK")

## Step 3: Anthropic API Test

In [ ]:
print("=" * 60)
print("STEP 3: ANTHROPIC API TEST")
print("=" * 60)
print("⚠️ This will likely fail due to insufficient credits - that's expected!")

try:
    import anthropic
    from config import Config
    import json
    
    if not Config.ANTHROPIC_API_KEY:
        print("❌ FAILED: No Anthropic API key found in configuration")
        print("💡 Check your .env file and ensure ANTHROPIC_API_KEY is set")
    else:
        print("🔑 Anthropic API key found in configuration")
        key_preview = Config.ANTHROPIC_API_KEY[:20] + "..." + Config.ANTHROPIC_API_KEY[-10:]
        print(f"   Key preview: {key_preview}")
        print(f"   Key length: {len(Config.ANTHROPIC_API_KEY)} characters")
        
        print("\n📡 Creating Anthropic client...")
        client = anthropic.Anthropic(api_key=Config.ANTHROPIC_API_KEY)
        print("✓ Client created successfully")
        
        print("\n🚀 Sending test request to Claude...")
        print("   Model: claude-3-haiku-20240307")
        print("   Max tokens: 50")
        print("   Message: 'Hello! Please respond with your name and confirm this API test is working.'")
        
        response = client.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=50,
            messages=[{
                "role": "user", 
                "content": "Hello! Please respond with your name and confirm this API test is working."
            }]
        )
        
        print("\n✅ SUCCESS: Anthropic API is working!")
        print("\n📝 Full Response Details:")
        print(f"   Model used: {response.model}")
        print(f"   Response ID: {response.id}")
        print(f"   Stop reason: {response.stop_reason}")
        
        print("\n💬 Claude's Response:")
        print(f"   '{response.content[0].text}'")
        
        print("\n📊 Token Usage:")
        print(f"   Input tokens: {response.usage.input_tokens}")
        print(f"   Output tokens: {response.usage.output_tokens}")
        print(f"   Total tokens: {response.usage.input_tokens + response.usage.output_tokens}")
        
        # Estimate cost (approximate)
        input_cost = response.usage.input_tokens * 0.00025 / 1000  # $0.25 per 1K tokens
        output_cost = response.usage.output_tokens * 0.00125 / 1000  # $1.25 per 1K tokens
        total_cost = input_cost + output_cost
        print(f"   Estimated cost: ${total_cost:.6f}")
        
        print("\n🎉 ANTHROPIC API TEST: PASSED")
        
except Exception as e:
    print(f"\n❌ FAILED: Anthropic API test failed")
    print(f"\n🔍 Error Details:")
    print(f"   Error type: {type(e).__name__}")
    print(f"   Error message: {str(e)}")
    
    # Parse different types of errors
    error_str = str(e).lower()
    
    if "credit balance" in error_str or "insufficient" in error_str:
        print("\n💡 DIAGNOSIS: Insufficient credits in Anthropic account")
        print("\n🔧 SOLUTION:")
        print("   1. Go to: https://console.anthropic.com/settings/billing")
        print("   2. Add at least $5-10 in credits")
        print("   3. Wait a few minutes for credits to be processed")
        print("   4. Re-run this test")
        print("\n📋 This is the EXPECTED error - nothing is broken!")
        
    elif "authentication" in error_str or "api key" in error_str:
        print("\n💡 DIAGNOSIS: API key authentication issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your .env file")
        print("   2. Verify ANTHROPIC_API_KEY is correct")
        print("   3. Get new key from: https://console.anthropic.com/settings/keys")
        
    elif "network" in error_str or "connection" in error_str:
        print("\n💡 DIAGNOSIS: Network connectivity issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your internet connection")
        print("   2. Try again in a few minutes")
        print("   3. Check if corporate firewall is blocking the request")
        
    else:
        print(f"\n💡 DIAGNOSIS: Unexpected error")
        print(f"\n🔧 SOLUTION:")
        print(f"   1. Try again in a few minutes")
        print(f"   2. Check Anthropic status page")
        print(f"   3. Verify your API key is valid")
    
    print("\n❌ ANTHROPIC API TEST: FAILED (but this might be expected)")

## Step 4: Langfuse Test

In [ ]:
print("=" * 60)
print("STEP 4: LANGFUSE CONNECTION TEST")
print("=" * 60)

try:
    from langfuse import Langfuse
    from config import Config
    from datetime import datetime
    import time
    
    if not Config.LANGFUSE_PUBLIC_KEY or not Config.LANGFUSE_SECRET_KEY:
        print("❌ FAILED: Langfuse keys not found in configuration")
        print("💡 Check your .env file for LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY")
    else:
        print("🔑 Langfuse keys found in configuration")
        pub_preview = Config.LANGFUSE_PUBLIC_KEY[:15] + "..."
        sec_preview = Config.LANGFUSE_SECRET_KEY[:15] + "..."
        print(f"   Public key: {pub_preview} ({len(Config.LANGFUSE_PUBLIC_KEY)} chars)")
        print(f"   Secret key: {sec_preview} ({len(Config.LANGFUSE_SECRET_KEY)} chars)")
        print(f"   Host: {Config.LANGFUSE_HOST}")
        
        print("\n📡 Creating Langfuse client...")
        langfuse = Langfuse(
            public_key=Config.LANGFUSE_PUBLIC_KEY,
            secret_key=Config.LANGFUSE_SECRET_KEY,
            host=Config.LANGFUSE_HOST
        )
        print("✓ Langfuse client created successfully")
        
        print("\n🧪 Testing authentication with auth check...")
        try:
            auth_result = langfuse.auth_check()
            print("✓ Authentication successful")
        except Exception as auth_error:
            print(f"⚠️ Auth check failed: {auth_error}")
        
        print("\n🎯 Creating a test trace for tracking...")
        test_timestamp = datetime.now().isoformat()
        
        # Create a trace first
        trace = langfuse.start_span(
            name="setup_verification_trace",
            metadata={
                "setup_phase": "initial_testing",
                "components_tested": ["config", "dependencies", "anthropic", "langfuse"],
                "timestamp": test_timestamp,
                "test_type": "setup_verification",
                "project": "teamcenter_ai_agent",
                "version": "1.0.0"
            }
        )
        print("✓ Test trace created successfully")
        
        print("\n📊 Trace Details:")
        if hasattr(trace, 'id'):
            print(f"   Trace ID: {trace.id}")
        print(f"   Test name: setup_verification_trace")
        print(f"   Timestamp: {test_timestamp}")
        
        print("\n🧪 Creating test generation within trace...")
        generation = langfuse.start_generation(
            name="setup_test_detailed",
            model="test-model-v1", 
            input="This is a setup test to verify Langfuse connectivity and tracking capabilities",
            metadata={
                "test_type": "setup_verification",
                "test_timestamp": test_timestamp,
                "project": "teamcenter_ai_agent",
                "version": "1.0.0"
            }
        )
        
        # Complete the generation
        generation.end(
            output="Setup test completed successfully - Langfuse is working properly"
        )
        print("✓ Test generation created and completed")
        
        print("\n⏱️ Waiting for Langfuse to process...")
        time.sleep(2)  # Give Langfuse time to process
        
        # Complete the trace
        trace.end()
        print("✓ Trace completed successfully")
        
        print("\n✅ SUCCESS: Langfuse connection and tracking working!")
        print("\n📋 What was tested:")
        print("   ✓ API key authentication")
        print("   ✓ Client initialization")
        print("   ✓ Authentication check")
        print("   ✓ Trace creation")
        print("   ✓ Generation creation and completion")
        print("   ✓ Metadata attachment")
        
        print("\n🌐 You can view these test entries in your Langfuse dashboard:")
        print(f"   Dashboard: {Config.LANGFUSE_HOST}")
        
        print("\n🎉 LANGFUSE TEST: PASSED")
        
except Exception as e:
    print(f"\n❌ FAILED: Langfuse test failed")
    print(f"\n🔍 Error Details:")
    print(f"   Error type: {type(e).__name__}")
    print(f"   Error message: {str(e)}")
    
    # Parse different types of errors
    error_str = str(e).lower()
    
    if "unauthorized" in error_str or "authentication" in error_str:
        print("\n💡 DIAGNOSIS: API key authentication issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your .env file")
        print("   2. Verify LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY are correct")
        print("   3. Get new keys from your Langfuse project settings")
        print("   4. Ensure you're using the right host URL")
        
    elif "connection" in error_str or "network" in error_str:
        print("\n💡 DIAGNOSIS: Network connectivity issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your internet connection")
        print("   2. Verify the Langfuse host URL is correct")
        print("   3. Check if corporate firewall is blocking the request")
        
    elif "timeout" in error_str:
        print("\n💡 DIAGNOSIS: Request timeout")
        print("\n🔧 SOLUTION:")
        print("   1. Try again - this might be temporary")
        print("   2. Check Langfuse service status")
        print("   3. Verify your host URL is correct")
        
    else:
        print(f"\n💡 DIAGNOSIS: Unexpected error")
        print(f"\n🔧 SOLUTION:")
        print(f"   1. Check Langfuse documentation")
        print(f"   2. Verify your project settings")
        print(f"   3. Try again in a few minutes")
    
    print("\n🔍 Full traceback:")
    import traceback
    traceback.print_exc()
    
    print("\n❌ LANGFUSE TEST: FAILED")

## Step 5: OpenAI API Test

In [ ]:
print("=" * 60)
print("STEP 5: OPENAI API TEST")
print("=" * 60)
print("🔧 Testing fallback/comparison API - OpenAI GPT models")

try:
    import openai
    from config import Config
    import json
    
    if not Config.OPENAI_API_KEY:
        print("○ SKIPPED: No OpenAI API key found in configuration")
        print("💡 This is optional - OpenAI is used as fallback/comparison model")
        print("🔧 To test OpenAI:")
        print("   1. Get API key from: https://platform.openai.com/account/api-keys")
        print("   2. Add OPENAI_API_KEY to your .env file")
        print("   3. Re-run this test")
        print("\n✅ OPENAI TEST: SKIPPED (OPTIONAL)")
    else:
        print("🔑 OpenAI API key found in configuration")
        key_preview = Config.OPENAI_API_KEY[:20] + "..." + Config.OPENAI_API_KEY[-10:]
        print(f"   Key preview: {key_preview}")
        print(f"   Key length: {len(Config.OPENAI_API_KEY)} characters")
        
        print("\n📡 Creating OpenAI client...")
        client = openai.OpenAI(api_key=Config.OPENAI_API_KEY)
        print("✓ OpenAI client created successfully")
        
        print("\n🚀 Sending test request to GPT...")
        print("   Model: gpt-3.5-turbo")
        print("   Max tokens: 50")
        print("   Message: 'Hello! Please confirm this OpenAI API test is working and tell me your model name.'")
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            max_tokens=50,
            messages=[{
                "role": "user", 
                "content": "Hello! Please confirm this OpenAI API test is working and tell me your model name."
            }],
            temperature=0.1
        )
        
        print("\n✅ SUCCESS: OpenAI API is working!")
        print("\n📝 Full Response Details:")
        print(f"   Model used: {response.model}")
        print(f"   Response ID: {response.id}")
        print(f"   Object type: {response.object}")
        print(f"   Finish reason: {response.choices[0].finish_reason}")
        
        print("\n💬 GPT's Response:")
        print(f"   '{response.choices[0].message.content}'")
        
        print("\n📊 Token Usage:")
        usage = response.usage
        print(f"   Prompt tokens: {usage.prompt_tokens}")
        print(f"   Completion tokens: {usage.completion_tokens}")
        print(f"   Total tokens: {usage.total_tokens}")
        
        # Estimate cost for GPT-3.5-turbo (approximate)
        prompt_cost = usage.prompt_tokens * 0.0015 / 1000  # $0.0015 per 1K tokens
        completion_cost = usage.completion_tokens * 0.002 / 1000  # $0.002 per 1K tokens
        total_cost = prompt_cost + completion_cost
        print(f"   Estimated cost: ${total_cost:.6f}")
        
        print("\n🎯 Testing model capabilities...")
        
        # Test with a more complex request
        print("   Testing reasoning with a simple math problem...")
        math_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            max_tokens=30,
            messages=[{
                "role": "user", 
                "content": "What is 15 + 27? Please show the calculation."
            }],
            temperature=0
        )
        
        print(f"   Math response: '{math_response.choices[0].message.content}'")
        print(f"   Additional tokens used: {math_response.usage.total_tokens}")
        
        print("\n🎉 OPENAI API TEST: PASSED")
        print("\n📋 Capabilities verified:")
        print("   ✓ API authentication")
        print("   ✓ Chat completion generation")
        print("   ✓ Token usage tracking")
        print("   ✓ Response formatting")
        print("   ✓ Basic reasoning/math")
        
except Exception as e:
    print(f"\n❌ FAILED: OpenAI API test failed")
    print(f"\n🔍 Error Details:")
    print(f"   Error type: {type(e).__name__}")
    print(f"   Error message: {str(e)}")
    
    # Parse different types of errors
    error_str = str(e).lower()
    
    if "invalid api key" in error_str or "authentication" in error_str:
        print("\n💡 DIAGNOSIS: API key authentication issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your .env file")
        print("   2. Verify OPENAI_API_KEY is correct")
        print("   3. Get new key from: https://platform.openai.com/account/api-keys")
        print("   4. Ensure you have credits in your OpenAI account")
        
    elif "insufficient" in error_str or "quota" in error_str or "billing" in error_str:
        print("\n💡 DIAGNOSIS: Insufficient credits or quota exceeded")
        print("\n🔧 SOLUTION:")
        print("   1. Check your OpenAI billing: https://platform.openai.com/account/billing")
        print("   2. Add credits to your account")
        print("   3. Check your usage limits")
        
    elif "rate limit" in error_str:
        print("\n💡 DIAGNOSIS: Rate limit exceeded")
        print("\n🔧 SOLUTION:")
        print("   1. Wait a minute and try again")
        print("   2. Check your rate limits in OpenAI dashboard")
        print("   3. Consider upgrading your plan for higher limits")
        
    elif "connection" in error_str or "network" in error_str:
        print("\n💡 DIAGNOSIS: Network connectivity issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your internet connection")
        print("   2. Try again in a few minutes")
        print("   3. Check if corporate firewall is blocking OpenAI")
        
    else:
        print(f"\n💡 DIAGNOSIS: Unexpected error")
        print(f"\n🔧 SOLUTION:")
        print(f"   1. Check OpenAI status page")
        print(f"   2. Verify your API key is valid")
        print(f"   3. Try again in a few minutes")
    
    print("\n🔍 Full traceback:")
    import traceback
    traceback.print_exc()
    
    print("\n❌ OPENAI API TEST: FAILED")

## Step 6: Google Gemini API Test

In [ ]:
print("=" * 60)
print("STEP 6: GOOGLE GEMINI API TEST")
print("=" * 60)
print("🔧 Testing fallback/comparison API - Google Gemini models")

try:
    import google.generativeai as genai
    from config import Config
    import json
    
    if not Config.GOOGLE_API_KEY:
        print("○ SKIPPED: No Google API key found in configuration")
        print("💡 This is optional - Google Gemini is used as fallback/comparison model")
        print("🔧 To test Google Gemini:")
        print("   1. Get API key from: https://console.cloud.google.com/apis/credentials")
        print("   2. Enable the Generative AI API")
        print("   3. Add GOOGLE_API_KEY to your .env file")
        print("   4. Re-run this test")
        print("\n✅ GOOGLE GEMINI TEST: SKIPPED (OPTIONAL)")
    else:
        print("🔑 Google API key found in configuration")
        key_preview = Config.GOOGLE_API_KEY[:20] + "..."
        print(f"   Key preview: {key_preview}")
        print(f"   Key length: {len(Config.GOOGLE_API_KEY)} characters")
        
        print("\n📡 Configuring Google Generative AI...")
        genai.configure(api_key=Config.GOOGLE_API_KEY)
        print("✓ Google AI client configured successfully")
        
        print("\n🔍 Getting available models...")
        try:
            models = list(genai.list_models())
            available_models = [m.name for m in models if 'generateContent' in m.supported_generation_methods]
            print(f"   Found {len(available_models)} available models")
            if available_models:
                print(f"   First few models: {available_models[:3]}")
        except Exception as model_error:
            print(f"   Could not list models: {model_error}")
            available_models = ['models/gemini-1.5-pro', 'models/gemini-pro']  # fallback
        
        print("\n🚀 Sending test request to Gemini...")
        # Try the most current model first, fallback to older version
        if "models/gemini-1.5-pro" in available_models:
            model_name = "gemini-1.5-pro"
        elif "models/gemini-pro" in available_models:
            model_name = "gemini-pro"
        else:
            model_name = "gemini-1.5-pro"  # Default to current version
            
        print(f"   Model: {model_name}")
        print("   Message: 'Hello! Please confirm this Google Gemini API test is working and tell me about your capabilities.'")

        model = genai.GenerativeModel(model_name)
        response = model.generate_content(
            "Hello! Please confirm this Google Gemini API test is working and tell me about your capabilities.",
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=100,
                temperature=0.1
            )
        )
        
        print("\n✅ SUCCESS: Google Gemini API is working!")
        print("\n📝 Full Response Details:")
        print(f"   Model used: {model_name}")
        
        # Check if response has the expected attributes
        if hasattr(response, 'text'):
            print(f"   Response text available: Yes")
            print(f"   Response length: {len(response.text)} characters")
        
        if hasattr(response, 'candidates'):
            print(f"   Number of candidates: {len(response.candidates)}")
            if response.candidates:
                candidate = response.candidates[0]
                if hasattr(candidate, 'finish_reason'):
                    print(f"   Finish reason: {candidate.finish_reason}")
                if hasattr(candidate, 'safety_ratings'):
                    print(f"   Safety ratings: {len(candidate.safety_ratings)} categories")
        
        print("\n💬 Gemini's Response:")
        response_text = response.text if hasattr(response, 'text') else str(response)
        print(f"   '{response_text[:200]}{'...' if len(response_text) > 200 else ''}'")
        
        print("\n🎯 Testing model capabilities...")
        
        # Test with a reasoning task
        print("   Testing reasoning with a word problem...")
        reasoning_response = model.generate_content(
            "If a train travels 60 mph and needs to go 180 miles, how long will it take? Show your calculation.",
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=80,
                temperature=0
            )
        )
        
        reasoning_text = reasoning_response.text if hasattr(reasoning_response, 'text') else str(reasoning_response)
        print(f"   Reasoning response: '{reasoning_text[:150]}{'...' if len(reasoning_text) > 150 else ''}'")
        
        # Test with a creative task
        print("   Testing creative writing...")
        creative_response = model.generate_content(
            "Write a haiku about artificial intelligence in exactly 3 lines.",
            generation_config=genai.types.GenerationConfig(
                max_output_tokens=50,
                temperature=0.7
            )
        )
        
        creative_text = creative_response.text if hasattr(creative_response, 'text') else str(creative_response)
        print(f"   Creative response: '{creative_text}'")
        
        print("\n🎉 GOOGLE GEMINI API TEST: PASSED")
        print("\n📋 Capabilities verified:")
        print("   ✓ API authentication")
        print("   ✓ Content generation")
        print("   ✓ Model configuration")
        print("   ✓ Response parsing")
        print("   ✓ Reasoning tasks")
        print("   ✓ Creative tasks")
        print("   ✓ Safety filtering")
        
except ImportError as import_error:
    print(f"\n❌ FAILED: Google AI library not installed")
    print(f"Error: {import_error}")
    print("\n🔧 SOLUTION:")
    print("   1. Install Google AI library: pip install google-generativeai")
    print("   2. Re-run this test")
    print("\n❌ GOOGLE GEMINI TEST: FAILED (MISSING LIBRARY)")
    
except Exception as e:
    print(f"\n❌ FAILED: Google Gemini API test failed")
    print(f"\n🔍 Error Details:")
    print(f"   Error type: {type(e).__name__}")
    print(f"   Error message: {str(e)}")
    
    # Parse different types of errors
    error_str = str(e).lower()
    
    if "api key" in error_str or "authentication" in error_str or "invalid" in error_str:
        print("\n💡 DIAGNOSIS: API key authentication issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your .env file")
        print("   2. Verify GOOGLE_API_KEY is correct")
        print("   3. Get new key from: https://console.cloud.google.com/apis/credentials")
        print("   4. Ensure Generative AI API is enabled in your project")
        
    elif "quota" in error_str or "limit" in error_str or "billing" in error_str:
        print("\n💡 DIAGNOSIS: Quota exceeded or billing issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your Google Cloud billing: https://console.cloud.google.com/billing")
        print("   2. Enable billing for your project")
        print("   3. Check your API quotas and limits")
        print("   4. Wait if you've hit rate limits")
        
    elif "permission" in error_str or "forbidden" in error_str:
        print("\n💡 DIAGNOSIS: Permissions issue")
        print("\n🔧 SOLUTION:")
        print("   1. Enable the Generative AI API in Google Cloud Console")
        print("   2. Check your project permissions")
        print("   3. Ensure your API key has the correct scopes")
        
    elif "connection" in error_str or "network" in error_str:
        print("\n💡 DIAGNOSIS: Network connectivity issue")
        print("\n🔧 SOLUTION:")
        print("   1. Check your internet connection")
        print("   2. Try again in a few minutes")
        print("   3. Check if corporate firewall is blocking Google APIs")
        
    else:
        print(f"\n💡 DIAGNOSIS: Unexpected error")
        print(f"\n🔧 SOLUTION:")
        print(f"   1. Check Google Cloud status page")
        print(f"   2. Verify your API key and project settings")
        print(f"   3. Try again in a few minutes")
    
    print("\n🔍 Full traceback:")
    import traceback
    traceback.print_exc()
    
    print("\n❌ GOOGLE GEMINI API TEST: FAILED")

## Step 7: Embedding Model Test

In [ ]:
print("=" * 60)
print("STEP 7: EMBEDDING MODEL TEST")
print("=" * 60)
print("NOTE: May take time to download model first time")

try:
    from sentence_transformers import SentenceTransformer
    from config import Config
    import numpy as np
    
    print(f"Loading model: {Config.EMBEDDING_MODEL}")
    model = SentenceTransformer(Config.EMBEDDING_MODEL)
    
    test_texts = [
        "How do I configure user roles in Teamcenter Easy Plan?",
        "What are the workflow setup procedures?",
        "User management and permissions configuration"
    ]
    
    print("Encoding test texts...")
    embeddings = model.encode(test_texts)
    
    similarity = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    
    print("SUCCESS: Embedding model working!")
    print(f"Embedding dimension: {embeddings.shape[1]}")
    print(f"Number of test embeddings: {embeddings.shape[0]}")
    print(f"Similarity between texts: {similarity:.4f}")
    
except Exception as e:
    print(f"FAILED: Embedding test failed: {e}")

## Step 8: FAISS Vector Database Test

In [ ]:
print("=" * 60)
print("STEP 8: FAISS VECTOR DATABASE TEST")
print("=" * 60)

try:
    import faiss
    import numpy as np
    
    dimension = 384
    index = faiss.IndexFlatL2(dimension)
    
    print(f"SUCCESS: FAISS index created (dimension: {dimension})")
    
    num_vectors = 5
    test_vectors = np.random.random((num_vectors, dimension)).astype('float32')
    index.add(test_vectors)
    
    print(f"SUCCESS: Added {num_vectors} test vectors")
    
    query_vector = np.random.random((1, dimension)).astype('float32')
    distances, indices = index.search(query_vector, k=3)
    
    print("SUCCESS: Vector search successful!")
    print(f"Found {len(indices[0])} results")
    print(f"Index size: {index.ntotal} vectors")
    
except Exception as e:
    print(f"FAILED: FAISS test failed: {e}")

## Step 9: Final Summary

In [ ]:
print("=" * 60)
print("FINAL SETUP STATUS SUMMARY")
print("=" * 60)

print("🎯 CORE COMPONENTS (Required for basic functionality):")
print("   ✅ Configuration Loading - Should PASS")
print("   ✅ Dependencies Installation - Should PASS") 
print("   ❌ Anthropic API (Primary) - Will FAIL (needs credits)")
print("   ✅ Langfuse Tracking - Should PASS")
print("   ✅ Embedding Model - Should PASS")
print("   ✅ FAISS Vector Database - Should PASS")

print("\n🔧 FALLBACK/COMPARISON APIS (Optional):")
print("   ○ OpenAI GPT Models - Optional (for model comparison)")
print("   ○ Google Gemini - Optional (for model comparison)")

print("\n📊 EXPECTED OUTCOMES:")

print("\n🟢 SHOULD PASS (5/6 core components):")
print("   • Configuration: All settings loaded, API keys detected")
print("   • Dependencies: All packages installed and importable")
print("   • Langfuse: Connection established, test tracking created")
print("   • Embeddings: Model downloaded and encoding working")
print("   • FAISS: Vector index created and search functional")

print("\n🔴 EXPECTED TO FAIL (1/6 core components):")
print("   • Anthropic API: 'Insufficient credits' error")
print("     └─ This is NORMAL and EXPECTED")
print("     └─ Solution: Add $5-10 credits to Anthropic account")

print("\n⭕ OPTIONAL TESTS:")
print("   • OpenAI: Will SKIP if no key provided (that's fine)")
print("   • Google Gemini: Will SKIP if no key provided (that's fine)")

print("\n🎉 SUCCESS CRITERIA:")
print("   📈 5 out of 6 core tests passing = EXCELLENT")
print("   📈 Only Anthropic failing due to credits = EXPECTED")
print("   📈 Optional APIs skipped = PERFECTLY FINE")

print("\n🚀 NEXT STEPS AFTER TESTING:")

print("\n1. 💳 ADD ANTHROPIC CREDITS (PRIORITY 1):")
print("   • Go to: https://console.anthropic.com/settings/billing")
print("   • Add minimum $5-10 credits")
print("   • Wait 2-3 minutes for processing")
print("   • Re-run Step 3 to verify")

print("\n2. 📁 PREPARE FOR DEVELOPMENT:")
print("   • Add sample documents to 'documents/' folder")
print("   • Review TODO.md for implementation roadmap")
print("   • Start with Phase 2: Core Data Models")

print("\n3. 🧪 OPTIONAL: TEST FALLBACK APIS:")
print("   • Get OpenAI key for model comparison")
print("   • Get Google key for additional model options")
print("   • These are NOT required for core functionality")

print("\n✨ YOUR SYSTEM STATUS:")
if_all_pass = "🎉 READY FOR DEVELOPMENT!"
if_anthropic_fails = "🎯 ALMOST READY - Just need Anthropic credits!"

print(f"   Expected status: {if_anthropic_fails}")
print("   Your RAG system foundation is solid!")

from datetime import datetime
print(f"\n⏰ Test completed at: {datetime.now()}")
print("\n" + "=" * 60)